In [19]:
import pandas as pd 
import ast 
import numpy as np 
import matplotlib.pyplot as plt
from collections import defaultdict

In [25]:
db_fluff = pd.read_csv('../data/no_fluffs.csv')
db_so = pd.read_csv('../data/so.csv')
db_so.rename(columns={
    "job_type": "Time",
    "role": "Role",
    'company_size': 'Size',
    "company_type": "Company",
    "tags": "Tags",
    'industry': 'Industry',
    'experience': 'Experience',
    'location': 'Locations',
    'salary': "Salary",
    'remote': "Remote"
},
             inplace=True)

def to_eur(money, currency):
    if currency == 'EUR' or currency == '€':
        return money
    elif currency == 'USD' or currency == '$':
        return money / 1.08
    elif currency == 'A$' or currency == 'AUD':
        return money / 1.80
    elif currency == 'PLN':
        return money / 4.58
    elif currency == 'kr':
        return money / 11.00
    elif currency == 'GBP' or currency == '£':
        return money / 0.88
    elif currency == 'CHF':
        return money / 1.06
    elif currency == 'CAD' or currency == 'C$':
        return money / 1.53
    elif currency == 'HUF':
        return money / 367.93
    elif currency == 'CZK':
        return money / 27.78
    elif currency == '₹' or currency == 'JPY':
        return money / 117.25
    else:
        None

def to_annual_eur(money, currency):
    money = float(money)
    if currency == 'PLN':

        # Per hour
        if money <= 500:
            money *= 160

        # Per day
        if money > 500 and money <= 2000:
            money *= 20

        # To year
        money *= 12

    if currency == '$':
        # To year salary
        if money < 1000:
            money *= 160 * 12

    return to_eur(money, currency)

def match_salaries(salary_str):
    if not salary_str:
        return 0.0, 0.0, None
    if isinstance(salary_str, float):
        return 0.0, 0.0, None
    if '-' in salary_str:
        res = salary_str.strip().split(' ')
        if len(res) == 4:
            min_s, _, max_s, currency = res 
        elif len(res) == 5:
            min_s, _, max_s, currency, _ = res 
    else:
        min_s, currency = salary_str.strip().split(' ')
        max_s = min_s
        
    min_s = min_s.lower()
    max_s = max_s.lower()
    if 'k' in min_s:
        min_s = float(min_s.replace('k', '')) * 1e3
    elif 'm' in min_s:
        min_s = float(min_s.replace('m', '')) * 1e6
    else:
        min_s = int(min_s)
    if 'k' in max_s:
        max_s = float(max_s.replace('k', '')) * 1e3
    elif 'm' in max_s:
        max_s = float(max_s.replace('m', '')) * 1e6
    else:
        max_s = float(max_s)

    return [min_s, max_s, currency]


res = db_fluff['Salary'].apply(match_salaries).values
min_s, max_s, curr_s, min_annual_eur, max_annual_eur = [], [], [], [], []
for mins, maxs, currs in res:
    min_s.append(mins)
    max_s.append(maxs)
    curr_s.append(currs)
    min_annual_eur.append( to_annual_eur(mins, currs) )
    max_annual_eur.append( to_annual_eur(maxs, currs) )

print(len(curr_s), len(min_s), len(max_s), len(db_fluff))
db_fluff['Min_Salary'] = min_s
db_fluff['Max_Salary'] = max_s
db_fluff['Currency'] = curr_s
db_fluff['Min_Annual_Eur_Salary'] = min_annual_eur
db_fluff['Max_Annual_Eur_Salary'] = max_annual_eur

null_tuple = (0., 0., None)

def match_salaries(salary_str):
    if salary_str == 'None':
        return null_tuple
    if isinstance(salary_str, float):
        return null_tuplee
    for syf in ['MXN', 'SGD', 'NT']:
        if salary_str.find(syf) != -1:
            return null_tuple
    currency = None 
    
    for repl_val in ['A$', 'C$', '$', '€', 'CHF', 'PLN', 'NZD',
                     'DKK', 'SEK', 'INR', 'GBP', '£',
                    'CAD', 'kr', 'zł', 'USD', '₹', '¥', 
                    'Php', 'RM']:
        if not currency and repl_val in salary_str:
            currency = repl_val
        salary_str = salary_str.replace(repl_val, '')
        
    salary_str = salary_str.strip()
#     print(salary_str, currency)
    
    if '-' in salary_str:
        res = salary_str.strip().split(' ')
        if currency:
            min_s, max_s = res[0], res[2]
        elif len(res) == 3:
            min_s, _, max_s = res 
        elif len(res) == 4:
            min_s, _, max_s, currency = res 
        elif len(res) == 5:
            min_s, _, max_s, currency, _ = res 

    else:
        min_s, currency = salary_str.strip().split(' ')
        max_s = min_s
    
    min_s = min_s.lower()
    max_s = max_s.lower()

    if 'k' in min_s:
        min_s = float(min_s.replace('k', '')) * 1e3
    elif 'm' in min_s:
        min_s = float(min_s.replace('m', '')) * 1e6
    else:
        min_s = int(min_s)
    if 'k' in max_s:
        max_s = float(max_s.replace('k', '')) * 1e3
    elif 'm' in max_s:
        max_s = float(max_s.replace('m', '')) * 1e6
    else:
        max_s = float(max_s)

    return [min_s, max_s, currency]


res = db_so['Salary'].apply(match_salaries).values
min_s, max_s, curr_s, min_annual_eur, max_annual_eur = [], [], [], [], []
for mins, maxs, currs in res:
    min_s.append(mins)
    max_s.append(maxs)
    curr_s.append(currs)
    min_annual_eur.append( to_annual_eur(mins, currs) )
    max_annual_eur.append( to_annual_eur(maxs, currs) )

print(len(curr_s), len(min_s), len(max_s), len(db_fluff))
db_so['Min_Salary'] = min_s
db_so['Max_Salary'] = max_s
db_so['Currency'] = curr_s
db_so['Min_Annual_Eur_Salary'] = min_annual_eur
db_so['Max_Annual_Eur_Salary'] = max_annual_eur


def filter_tags(str_tag_list):
    tag_list = ast.literal_eval(str_tag_list)
    if not tag_list:
        return tag_list
    return [x.lower().strip() for x in tag_list]


db_fluff['Tags'] = db_fluff['Tags'].apply(filter_tags)
db_so['Tags'] = db_so['Tags'].apply(filter_tags)

1074 1074 1074 1074
12809 12809 12809 1074


In [26]:
db_fluff.head()

,Role,Tags,Additional,Salary,Locations,Min_Salary,Max_Salary,Currency,Min_Annual_Eur_Salary,Max_Annual_Eur_Salary
0,React Frontend Developer,"[react, javascript, css, git, rest, html5, nod...","{' Job profile ': ' Mainly new features ', ' S...",600k - 1.2M HUF,Budapest,600000.0,1200000.0,HUF,1630.744979,3261.489957
1,Front End Developer,"[angular, javascript, css, html, react, design...","{' Job profile ': ' Mainly new features ', ' S...",8000 - 15000 PLN,Kraków,8000.0,15000.0,PLN,20960.698690,39301.310044
2,Frontend Developer,"[javascript, jquery, angular, css, html, redux...","{' Job profile ': ' Mainly new features ', ' S...",900 - 1300 PLN,"Kraków, Katowice",900.0,1300.0,PLN,47161.572052,68122.270742
3,Python Developer,"[python, django, postgresql, experience in dev...","{' Start ': ' 2020-05-01 ', ' Contract duratio...",12000 - 14000 PLN,Warszawa,12000.0,14000.0,PLN,31441.048035,36681.222707
4,Mid/Senior Java Developer (with English),"[java, spring, hibernate, javascript, angular,...","{' Job profile ': ' Mainly new features ', ' S...",8000 - 16000 PLN,Wrocław,8000.0,16000.0,PLN,20960.698690,41921.397380


In [28]:
db_so.head()

,Time,Experience,Role,Industry,Size,Company,Salary,Remote,Locations,Tags,Min_Salary,Max_Salary,Currency,Min_Annual_Eur_Salary,Max_Annual_Eur_Salary
0,Full-time,"Mid-Level, Senior",None,"Online Media, Software Development / Engineeri...",1k–5k people,Private,None,None,"Amsterdam,Netherlands","[amazon-web-services, aws, cloud, go, python]",0.0,0.0,None,NaN,NaN
1,Full-time,"Mid-Level, Senior",DevOps,"Digital Health, Health Care, Healthcare",201–500 people,VC Funded,$130k - 150k (US Dollars),None,"Denver,CO","[python, google-cloud-platform, postgresql, do...",130000.0,150000.0,$,NaN,NaN
2,Full-time,None,Mobile Developer,None,None,None,None,None,"Pune,India","[reactjs, ios, swift]",0.0,0.0,None,NaN,NaN
3,Full-time,None,None,None,None,None,None,None,"MinatoCity,Japan","[amazon-web-services, go, php]",0.0,0.0,None,NaN,NaN
4,Full-time,None,"Backend Developer, Mobile Developer",None,None,None,None,None,"Mumbai,India","[android, java, rest]",0.0,0.0,None,NaN,NaN


In [29]:
db_so[db_so['Salary'] != 'None'].head()

,Time,Experience,Role,Industry,Size,Company,Salary,Remote,Locations,Tags,Min_Salary,Max_Salary,Currency,Min_Annual_Eur_Salary,Max_Annual_Eur_Salary
1,Full-time,"Mid-Level, Senior",DevOps,"Digital Health, Health Care, Healthcare",201–500 people,VC Funded,$130k - 150k (US Dollars),None,"Denver,CO","[python, google-cloud-platform, postgresql, do...",130000.0,150000.0,$,NaN,NaN
28,Full-time,"Mid-Level, Senior, Lead",Mobile Developer,"Communications, Enterprise Software, SaaS",1–10 people,VC Funded,$90k - 130k (US Dollars),On-siteandlimitedremote,"LosAngeles,CA",[],90000.0,130000.0,$,NaN,NaN
49,Full-time,"Mid-Level, Senior",None,"Computer Software, Mobile Application, Web Dev...",11–50 people,Private,$60k - 120k (US Dollars),None,"Kowloon,HongKong",[kotlin],60000.0,120000.0,$,NaN,NaN
60,Full-time,"Mid-Level, Senior",None,"Hardware Development, Software Development, So...",51–200 people,Private,€47k - 63k (Euros),On-siteandlimitedremote,"Eindhoven,Netherlands",[],47000.0,63000.0,€,NaN,NaN
95,Full-time,"Junior, Mid-Level, Senior",Mobile Developer,None,None,None,€50k - 62k (Euros),None,"Berlin,Deutschland","[ios, swift, blockchain, eos]",50000.0,62000.0,€,NaN,NaN


In [30]:
df_major = pd.concat([db_so, db_fluff])

In [31]:
df_major = df_major[df_major['Role'] != 'None']
df_major['Time'].fillna(value='Full-time', inplace=True)
df_major['Experience'].fillna(value='Mid-Level', inplace=True)

In [32]:
df_major.to_csv('../data/DATABASE.csv')
df_major.to_pickle('../data/DATABASE.pkl')

In [18]:
df_major[(df_major['Remote'] != 'None') & (~df_major['Remote'].isna())]

,Time,Experience,Role,Industry,Size,Company,Salary,Remote,Locations,Tags,Additional
28,Full-time,"Mid-Level, Senior, Lead",Mobile Developer,"Communications, Enterprise Software, SaaS",1–10 people,VC Funded,$90k - 130k (US Dollars),On-siteandlimitedremote,"LosAngeles,CA",[],NaN
70,Full-time,"Junior, Mid-Level",Frontend Developer,"Software Consulting, Software Development, Sof...",501–1k people,Private,None,On-siteandlimitedremote,"Aachen,Deutschland",[],NaN
76,Full-time,"Student, Mid-Level",System Administrator,"Agile Software Development, Enterprise Softwar...",201–500 people,Private,None,On-siteandlimitedremote,"Leinfelden-Echterdingen,Germany",[],NaN
114,Full-time,None,Backend Developer,None,None,None,$90k - 120k (US Dollars),Remote,Noofficelocation,"['python', 'scikit-learn', 'pytorch', 'docker'...",NaN
169,Full-time,"Mid-Level, Senior, Lead",Full Stack Developer,"Convenience, Food & Beverage, Retail",10k+ people,Public,None,Remote,"Zürich,Switzerland",[],NaN
...,...,...,...,...,...,...,...,...,...,...,...
12736,Full-time,Senior,DevOps,"Customer Data Platform, Legal Technology, Mach...",51–200 people,Private,None,On-siteandlimitedremote,"München,Germany",[],NaN
12740,Full-time,"Junior, Mid-Level, Senior",Backend Developer,Education,11–50 people,Private,£25k - 40k (British Pounds),On-siteandlimitedremote,"KingstonuponHull,UK",[],NaN
12760,Full-time,"Mid-Level, Senior",DevOps,"Agile Software Development, Healthcare, Medica...",10k+ people,Public,$80k - 110k (US Dollars),On-siteandlimitedremote,"Chicago,IL",[],NaN
12763,Full-time,"Mid-Level, Senior, Lead",Full Stack Developer,"Consulting, Information Technology, IT Consulting",201–500 people,Private,None,Remote,Noofficelocation,"['spring', 'java', 'oracle', 'guidewire']",NaN
